In [18]:
#!/usr/bin/env python3
"""
Emittance growth pipeline (robust, despiked, fit-based)
-------------------------------------------------------
- Cleans slot time series (median/MAD despike + optional Gaussian smoothing)
- Robust linear fits (Theil–Sen) for injection & stable windows
- Uses ONLY the fit values to compute growth between end of injection and start of stable
- Plots per-family growth for all requested fills

Recursion/Assertion safe: we fully replace FillInjectionsForBeam, never calling the original.
"""

# ===================== USER CONFIG =====================================
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress, theilslopes

fills = [10671, 10672, 10673, 10674, 10676, 10677, 10678, 10683,
         10688, 10689, 10690, 10691, 10696, 10697, 10700, 10701,
         10702, 10703, 10704, 10705, 10709, 10713, 10716, 10718,
         10721, 10731, 10732]

beam    = "B1"
ip      = "1"   # string
RAWDATA = Path('/eos/project/l/lhc-lumimod/LuminosityFollowUp/2025/rawdata')
FILLINFO= Path('/eos/project/l/lhc-lumimod/LuminosityFollowUp/2025/fills-info')
fbmodes = pd.read_parquet(FILLINFO / 'fills_and_bmodes_2025.parquet')

# PVs
int_var    = f"LHC.BCTFR.B6R4.{beam}:BUNCH_INTENSITY"
emitH_var  = f"LHC.BSRT.5R4.{beam}:BUNCH_EMITTANCE_H"
emitV_var  = f"LHC.BSRT.5R4.{beam}:BUNCH_EMITTANCE_V"
lev_pv     = "LHC.LUMISERVER:LumiLevelingIP1:Enable"
beta_pv    = "HX:BETASTAR_IP1"
xing_pv    = "LhcStateTracker:LHCBEAM:IP1-XING-H-MURAD:value"

# Parameters
INTENSITY_THRESHOLD = 1e10
TOL_MINUTES         = 5
DESPIKE_K           = 5.0
DESPIKE_WIN         = '5min'
GAUSS_SIGMA_PTS     = 0
ROBUST_FIT_FUNC     = 'theilsen'   # or 'ols'
# =======================================================================

# ---------------- MONKEY-PATCH (direct replacement) --------------------
import LHC_FillingPattern as _lfp
from pathlib import Path as _P

def _safe_concat_sort(dflist):
    if not dflist:
        return pd.DataFrame()
    df = pd.concat(dflist, sort=True)
    if not df.empty:
        df = df.sort_index()
        df = df[~df.index.duplicated(keep='first')]
    return df

def FillInjectionsForBeam_direct(fno, ddir, beam, threshold):
    """Direct parquet reader for bunch intensity, sorted & deduped."""
    root = _P(ddir) / f"HX:FILLN={fno}"
    col  = f"LHC.BCTFR.B6R4.{beam}:BUNCH_INTENSITY"
    dflist = []
    for pq in root.rglob('*.parquet'):
        try:
            tmp = pd.read_parquet(pq, columns=[col])
        except Exception:
            continue
        tmp.index = pd.to_datetime(tmp.index, utc=True, errors='coerce')
        dflist.append(tmp.dropna())
    return _safe_concat_sort(dflist)

# Patch
_lfp.FillInjectionsForBeam = FillInjectionsForBeam_direct
from LHC_FillingPattern import LHCFillingPattern
# -----------------------------------------------------------------------

# --------------------- UTILITIES ---------------------------------------

def load_series(pv: str, fno: int, root: pathlib.Path) -> pd.Series:
    parts = []
    fill_dir = root / f"HX:FILLN={fno}"
    for pq in fill_dir.rglob('*.parquet'):
        try:
            df = pd.read_parquet(pq, columns=[pv])
        except Exception:
            continue
        idx = pd.to_datetime(df.index, utc=True, errors='coerce')
        parts.append(pd.Series(df[pv].values, index=idx, name=pv).dropna())
    return pd.concat(parts, sort=True) if parts else pd.Series(dtype=float, name=pv)


def load_INJPHYS(fno: int, var: str) -> pd.Series:
    if fno not in fbmodes.index:
        return pd.Series(dtype=float)
    sub = fbmodes.loc[fno]
    if isinstance(sub, pd.DataFrame):
        rows = sub.query("BMODE=='INJPHYS'").sort_values('tsStart')
    else:
        rows = pd.DataFrame([sub]).query("BMODE=='INJPHYS'")
    if rows.empty:
        rows = sub.sort_values('tsStart').iloc[[0]] if isinstance(sub, pd.DataFrame) else pd.DataFrame([sub])
    t0 = pd.to_datetime(rows['tsStart'].iloc[0], utc=True)
    t1 = pd.to_datetime(rows['tsEnd'  ].iloc[0], utc=True)
    s = load_series(var, fno, RAWDATA)
    return s.sort_index().loc[t0:t1].dropna()


def phase_markers_from_beta_xing(ser_beta: pd.Series, ser_xing: pd.Series, xing_threshold: float = 120.0):
    if ser_beta.empty or ser_xing.empty:
        return pd.NaT, pd.NaT, pd.NaT
    runs = []
    t_prev, v_prev = ser_beta.index[0], ser_beta.iloc[0]
    run_start = t_prev
    for t, v in ser_beta.iloc[1:].items():
        if v != v_prev:
            runs.append((run_start, t_prev))
            run_start, v_prev = t, v
        t_prev = t
    runs.append((run_start, t_prev))
    t_beta_final = max(runs, key=lambda x: x[1] - x[0])[0]

    ca0   = ser_xing.iloc[0]
    leave = ser_xing.index[(ser_xing != ca0).argmax()] if (ser_xing != ca0).any() else ser_xing.index[-1]
    dip   = ser_xing.index[(ser_xing <= xing_threshold).argmax()] if (ser_xing <= xing_threshold).any() else ser_xing.index[-1]
    return t_beta_final, leave, dip


def extract_slot(ser: pd.Series, slot: int) -> pd.Series:
    return ser.map(lambda a: a[slot] if hasattr(a,'__len__') and len(a)>slot else np.nan).dropna()

# Cleaning

def despike(series: pd.Series, k: float = DESPIKE_K, win: str = DESPIKE_WIN) -> pd.Series:
    if series.empty:
        return series
    med = series.rolling(win, min_periods=1).median()
    mad = (series - med).abs().rolling(win, min_periods=1).median()
    z = (series - med) / mad.replace(0, np.nan)
    cleaned = series.mask(z.abs() > k)
    return cleaned.interpolate(limit_direction='both')

try:
    from scipy.ndimage import gaussian_filter1d
    def gaussian_smooth(series: pd.Series, sigma_pts: int = GAUSS_SIGMA_PTS) -> pd.Series:
        if sigma_pts <= 0 or series.empty:
            return series
        y = gaussian_filter1d(series.values, sigma=sigma_pts, mode='nearest')
        return pd.Series(y, index=series.index)
except Exception:
    def gaussian_smooth(series: pd.Series, sigma_pts: int = GAUSS_SIGMA_PTS) -> pd.Series:
        return series

# Robust fit

def robust_linfit(x: np.ndarray, y: np.ndarray):
    if ROBUST_FIT_FUNC == 'theilsen':
        slope, intercept, _, _ = theilslopes(y, x)
    else:
        r = linregress(x, y)
        slope, intercept = r.slope, r.intercept
    return slope, intercept

# ------------------ Families builder -----------------------------------

def build_families_for_fill(fillno: int, beam: str, ip: str, RAWDATA: Path,
                             long_gap_set=(32,63), small_gap=8,
                             F4_offset_after_long=6, F6_offset_before_small=26,
                             enforce_filled=True, intensity_threshold=None) -> dict:
    fpat = LHCFillingPattern(fillno, RAWDATA)
    if beam.upper() == 'B1':
        trains = fpat.bunchtrainsDF_b1.copy(); filled_slots = fpat.bunches_b1
    else:
        trains = fpat.bunchtrainsDF_b2.copy(); filled_slots = fpat.bunches_b2
    trains = trains.loc[trains['id'] != 0].sort_values('bid_first').reset_index(drop=True)
    orbit_len = 3564

    if intensity_threshold is not None:
        ser_int = load_series(int_var, fillno, RAWDATA)
        if ser_int.empty:
            arr_intensity = np.zeros(orbit_len)
        else:
            last_vals = np.asarray(ser_int.iloc[-1])
            if last_vals.shape[0] != orbit_len:
                tmp = np.zeros(orbit_len); n = min(orbit_len, last_vals.shape[0])
                tmp[:n] = last_vals[:n]; last_vals = tmp
            arr_intensity = last_vals
    else:
        arr_intensity = None

    filled_mask = np.zeros(orbit_len, dtype=bool); filled_mask[filled_slots] = True
    def _filter(arr: np.ndarray) -> np.ndarray:
        if arr.size == 0:
            return arr
        arr = arr.astype(int)
        if enforce_filled:
            arr = arr[filled_mask[arr]]
        if arr_intensity is not None and intensity_threshold is not None:
            arr = arr[arr_intensity[arr] >= intensity_threshold]
        return arr

    mask_long = trains['gap'].isin(long_gap_set)
    Family_1 = trains.loc[mask_long, 'bid_first'].to_numpy(int)

    next_start = trains['bid_first'].shift(-1).fillna(orbit_len).astype(int)
    trains['gap_after'] = next_start - trains['bid_last'] - 1
    Family_2 = trains.loc[trains['gap_after'] >= 31, 'bid_last'].to_numpy(int)

    lr = fpat.lrencounters[beam][f"ip{ip}"]
    peak_slots = []
    for _, row in trains.iterrows():
        bids = np.asarray(row['bids'], dtype=int)
        lr_train = lr[bids]
        first_idx = np.where(lr_train == lr_train.max())[0][0]
        peak_slots.append(bids[first_idx])
    Family_3 = np.asarray(peak_slots, dtype=int)

    Family_4 = (trains.loc[mask_long, 'bid_first'] + F4_offset_after_long).to_numpy(int)

    mask_gap_small_15 = (trains['gap'] == small_gap) & (trains['nbunches'] >= 15)
    cand_F5 = (trains.loc[mask_gap_small_15, 'bid_first'] + 14).to_numpy(int)
    last_F5 = trains.loc[mask_gap_small_15, 'bid_last'].to_numpy(int)
    Family_5 = cand_F5[cand_F5 <= last_F5]

    rows_gap8 = trains['gap'] == small_gap
    train_before_gap8 = trains.shift(1).loc[rows_gap8].dropna()
    mask_nbunch27 = train_before_gap8['nbunches'] >= (F6_offset_before_small + 1)
    Family_6 = (train_before_gap8.loc[mask_nbunch27, 'bid_first'] + F6_offset_before_small).to_numpy(int)

    Family_7 = trains.loc[rows_gap8, 'bid_first'].to_numpy(int)

    train_before_gap8_full = trains.shift(1).loc[rows_gap8].dropna()
    Family_8 = train_before_gap8_full['bid_last'].astype(int).to_numpy()

    return {
        'Family_1': _filter(Family_1),
        'Family_2': _filter(Family_2),
        'Family_3': _filter(Family_3),
        'Family_4': _filter(Family_4),
        'Family_5': _filter(Family_5),
        'Family_6': _filter(Family_6),
        'Family_7': _filter(Family_7),
        'Family_8': _filter(Family_8),
    }

# ---------------- Injection regression ---------------------------------

def one_fill_family_rates_reg(fno: int, beam: str, families: dict, threshold: float = INTENSITY_THRESHOLD) -> pd.DataFrame:
    I  = load_INJPHYS(fno, int_var)
    EH = load_INJPHYS(fno, emitH_var)
    EV = load_INJPHYS(fno, emitV_var)

    per = {"H_slope":{}, "H_int":{}, "V_slope":{}, "V_int":{}, "t0":{}}
    for fam, slots in families.items():
        for slot in slots:
            Ii = extract_slot(I, slot)
            above = Ii[Ii >= threshold]
            if above.empty:
                for k in ("H_slope","H_int","V_slope","V_int"):
                    per[k][slot] = np.nan
                per['t0'][slot] = pd.NaT
                continue
            t0 = above.index[0]
            per['t0'][slot] = t0

            Hs = extract_slot(EH, slot).loc[t0:].sort_index()
            Vs = extract_slot(EV, slot).loc[t0:].sort_index()
            Hs = gaussian_smooth(despike(Hs))
            Vs = gaussian_smooth(despike(Vs))
            if Hs.dropna().size < 2 or Vs.dropna().size < 2:
                for k in ("H_slope","H_int","V_slope","V_int"):
                    per[k][slot] = np.nan
                continue

            xh = (Hs.index - t0).total_seconds()/3600.0
            xv = (Vs.index - t0).total_seconds()/3600.0
            mh, bh = robust_linfit(xh.values, Hs.values)
            mv, bv = robust_linfit(xv.values, Vs.values)
            per['H_slope'][slot] = mh; per['H_int'][slot] = bh
            per['V_slope'][slot] = mv; per['V_int'][slot] = bv

    rows = []
    for fam, slots in families.items():
        def avg(arr):
            arr = [a for a in arr if not np.isnan(a)]
            return np.mean(arr) if arr else np.nan
        rows.append({
            'fill': fno,
            'family': fam,
            'H_rate_avg_reg': avg([per['H_slope'][s] for s in slots]),
            'H_int_avg_reg' : avg([per['H_int'][s]   for s in slots]),
            'V_rate_avg_reg': avg([per['V_slope'][s] for s in slots]),
            'V_int_avg_reg' : avg([per['V_int'][s]   for s in slots]),
        })
    return pd.DataFrame(rows)

# ---------------- Stable regression ------------------------------------

def compute_growth_raw_regression(fno: int, families: dict):
    ser_beta = load_series(beta_pv, fno, RAWDATA)
    ser_xing = load_series(xing_pv, fno, RAWDATA)
    t_beta_final, t_xing_begin, t_xing_end = phase_markers_from_beta_xing(ser_beta, ser_xing)

    ser_lev = load_series(lev_pv, fno, RAWDATA).astype(float).dropna()
    dlev    = ser_lev.diff().fillna(0)
    t_on    = dlev[dlev>0].index.min() or ser_lev.index.min()

    ser_H = load_series(emitH_var, fno, RAWDATA)
    ser_V = load_series(emitV_var, fno, RAWDATA)
    rawH  = {s: extract_slot(ser_H, s).sort_index() for fam in families for s in families[fam]}
    rawV  = {s: extract_slot(ser_V, s).sort_index() for fam in families for s in families[fam]}
    for s in rawH: rawH[s] = gaussian_smooth(despike(rawH[s]))
    for s in rawV: rawV[s] = gaussian_smooth(despike(rawV[s]))

    all_raw = pd.concat(list(rawH.values()) + list(rawV.values()), axis=1) if rawH else pd.DataFrame()
    t0_eff  = all_raw.loc[t_on:].dropna(how='all').index.min() if not all_raw.empty else t_on

    windows = []
    prev, hr = t_on, 1
    if pd.notna(t_beta_final):
        for mark in pd.date_range(prev + pd.Timedelta(hours=1), t_beta_final, freq='1h'):
            windows.append((f"{hr}h since lvl-on", prev, mark)); prev, hr = mark, hr+1
        if prev < t_beta_final:
            windows.append(("to β* final", prev, t_beta_final)); prev = t_beta_final
        windows.append(("β*→CA start", prev, t_xing_begin))
        windows.append(("CA start→end", t_xing_begin, t_xing_end))
    else:
        windows.append(("lvl-on→end", t_on, t_on + pd.Timedelta(hours=1)))

    tol = pd.Timedelta(minutes=TOL_MINUTES)
    rows = []
    for label, _, t1 in windows:
        t1b = pd.to_datetime(t1, utc=True).ceil('5min') if pd.notna(t1) else t_on
        if pd.notna(t_xing_end):
            t1b = min(t1b, t_xing_end.ceil('5min'))
        dt  = (t1b - t0_eff).total_seconds()/3600.0
        if dt <= 0:
            continue
        row = {'fill': fno, 'window': label}
        for fam, slots in families.items():
            sh, hints, sv, vints = [], [], [], []
            for s in slots:
                segH = rawH.get(s, pd.Series()).loc[t0_eff:t1b]
                if len(segH)>=2 and (segH.index[0]-t0_eff)<=tol:
                    xh = (segH.index - t0_eff).total_seconds()/3600
                    mh, bh = robust_linfit(xh.values, segH.values)
                    sh.append(mh); hints.append(bh)
                segV = rawV.get(s, pd.Series()).loc[t0_eff:t1b]
                    
                if len(segV)>=2 and (segV.index[0]-t0_eff)<=tol:
                    xv = (segV.index - t0_eff).total_seconds()/3600
                    mv, bv = robust_linfit(xv.values, segV.values)
                    sv.append(mv); vints.append(bv)
            row[f"{fam}_H_rate"]      = np.mean(sh)    if sh    else np.nan
            row[f"{fam}_H_intercept"] = np.mean(hints) if hints else np.nan
            row[f"{fam}_V_rate"]      = np.mean(sv)    if sv    else np.nan
            row[f"{fam}_V_intercept"] = np.mean(vints) if vints else np.nan
        rows.append(row)

    rows_df = pd.DataFrame(rows)
    if rows_df.empty:
        rows_df = pd.DataFrame(columns=['fill','window'])
    if 'fill' not in rows_df.columns:
        rows_df['fill'] = fno
    if 'window' not in rows_df.columns:
        rows_df['window'] = np.nan
    rows_df = rows_df.set_index(['fill','window'])
    return rows_df, t_on, t_beta_final, t0_eff

# ---------------- MAIN BUILD -------------------------------------------

tables_reg   = []
all_results_1= {}
phase_times  = {}

for fno in fills:
    fams = build_families_for_fill(fno, beam, ip, RAWDATA)
    tables_reg.append(one_fill_family_rates_reg(fno, beam, fams))
    stab_df, t_on, t_beta, t0_eff = compute_growth_raw_regression(fno, fams)
    all_results_1[fno] = stab_df
    phase_times[fno]   = (t_on, t_beta, t0_eff)

df_inj_rates_reg = (pd.concat(tables_reg, ignore_index=True)
                      .set_index(['fill','family'])
                      .sort_index())

# ---------------- GROWTH FROM FITS -------------------------------------

growth_records = []
for fno in fills:
    t_on, t_beta, t0_eff = phase_times.get(fno, (pd.NaT, pd.NaT, pd.NaT))
    if pd.isna(t_on) or pd.isna(t_beta):
        continue
    dt_gap_h = (t_beta - t_on).total_seconds()/3600.0
    if fno not in df_inj_rates_reg.index.get_level_values('fill'):
        continue
    inj_df = df_inj_rates_reg.xs(fno, level='fill')
    if inj_df.empty:
        continue
    try:
        stab_df = all_results_1[fno].xs('β*→CA start', level='window')
    except (KeyError, ValueError):
        continue

    for fam in inj_df.index:
        if fam not in stab_df.index:
            continue
        mH_inj = inj_df.loc[fam, 'H_rate_avg_reg']; bH_inj = inj_df.loc[fam, 'H_int_avg_reg']
        mV_inj = inj_df.loc[fam, 'V_rate_avg_reg']; bV_inj = inj_df.loc[fam, 'V_int_avg_reg']
        eps_H_end = bH_inj
        eps_V_end = bV_inj

        mH_st = stab_df.loc[fam, f'{fam}_H_rate'];      bH_st = stab_df.loc[fam, f'{fam}_H_intercept']
        mV_st = stab_df.loc[fam, f'{fam}_V_rate'];      bV_st = stab_df.loc[fam, f'{fam}_V_intercept']
        dt_st_h = (t_beta - t0_eff).total_seconds()/3600.0
        eps_H_start = bH_st + mH_st * dt_st_h
        eps_V_start = bV_st + mV_st * dt_st_h

        growth_records.append({
            'fill':   fno,
            'family': fam,
            'H_growth': (eps_H_start - eps_H_end)/dt_gap_h,
            'V_growth': (eps_V_start - eps_V_end)/dt_gap_h,
        })

df_growth = pd.DataFrame(growth_records).set_index(['fill','family']).sort_index()

# ---------------- PLOTS -------------------------------------------------
for plane in ('H','V'):
    col = f'{plane}_growth'
    if col not in df_growth.columns:
        continue
    piv = df_growth[col].unstack('fill')
    ax = piv.plot(figsize=(9,4), marker='o')
    ax.set_title(f'{plane}-plane Avg Emittance Growth (robust fits, despiked)')
    ax.set_xlabel('Family')
    ax.set_ylabel('Growth (units/hour)')
    ax.legend(title='Fill', bbox_to_anchor=(1.02,1), loc='upper left')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# df_growth is the final result


FileNotFoundError: /eos/project/l/lhc-lumimod/LuminosityFollowUp/2025/rawdata/HX:FILLN=10688/HX:BMODE=RAMP